In [1]:
import numpy as np

In [2]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

In [3]:
from NN.Layers.Dense import Dense
from NN.Activations.Sigmoid import Sigmoid
from NN.LossFunctions.MSE import MSE
from NN.Networks.Sequential import Sequential

In [4]:
## XOR Data
X = np.array([[0, 0], 
              [0, 1],
              [1, 0],
              [1, 1]])
X = X.reshape(4, 2, 1)
print(X)
print()

y = np.array([[0],
              [1],
              [1],
              [0]])
y = y.reshape(4, 1, 1)
print(y)

[[[0]
  [0]]

 [[0]
  [1]]

 [[1]
  [0]]

 [[1]
  [1]]]

[[[0]]

 [[1]]

 [[1]]

 [[0]]]


In [8]:
xor_model = Sequential([
        Dense(2, 3),
        Sigmoid(),
        Dense(3, 1),
        Sigmoid()
    ],
    loss_fn=MSE())

In [9]:
xor_model.train(X, y, epochs=10000, lr=0.1)

Completed 1/10000 epoch, loss = 0.36705357495611324
Completed 2/10000 epoch, loss = 0.36399926864162724
Completed 3/10000 epoch, loss = 0.3608959735369559
Completed 4/10000 epoch, loss = 0.35774796448286683
Completed 5/10000 epoch, loss = 0.3545600757424859
Completed 6/10000 epoch, loss = 0.3513377012595945
Completed 7/10000 epoch, loss = 0.34808678500704276
Completed 8/10000 epoch, loss = 0.34481380030304715
Completed 9/10000 epoch, loss = 0.3415257171980446
Completed 10/10000 epoch, loss = 0.3382299573432973
Completed 11/10000 epoch, loss = 0.3349343361416186
Completed 12/10000 epoch, loss = 0.3316469924405047
Completed 13/10000 epoch, loss = 0.32837630654115435
Completed 14/10000 epoch, loss = 0.32513080783840675
Completed 15/10000 epoch, loss = 0.3219190739451601
Completed 16/10000 epoch, loss = 0.31874962365447723
Completed 17/10000 epoch, loss = 0.3156308065158372
Completed 18/10000 epoch, loss = 0.312570692113155
Completed 19/10000 epoch, loss = 0.30957696230126697
Completed 20/

In [10]:
for x in X:
    print(xor_model.predict(x))

[[0.05322916]]
[[0.95357186]]
[[0.95387641]]
[[0.04805333]]
